# Financial Analysis 

This notebook contains :
- An automation of the financial data gathering process using the **Financial Modeling Prep API**.
- An Income Statement analysis of **IBM**, with visiualizations through **Plotly**.

#### Since the visualizations are made with plotly they are interactive, make sure to use a **nbviewer**  :
https://nbviewer.jupyter.org/github/BrahimMebrek/Finance_Python/blob/dd702c27c1f0bcb337c64f4600bb53aa5e6b360c/Financial_Analysis.ipynb

## Definitions

### Financial statements
Financial statements are written records that convey the business activities and the financial performance of a company. Financial statements are often audited by government agencies, accountants, firms, etc. to ensure accuracy and for tax, financing, or investing purposes. Financial statements include:

- Balance sheet ;
- Income statement ; (Our study will focus on it)
- Cash flow statement.

### Income statement
An income statement is one of the three important financial statements used for reporting a company's financial performance over a specific accounting period. 

Also known as the profit and loss statement or the statement of revenue and expense, the income statement primarily focuses on the company’s revenues and expenses during a particular period.

### Ticker symbol
A ticker symbol or stock symbol is an abbreviation used to uniquely identify publicly traded shares of a particular stock on a particular stock market.

## 1 Data Gathering and Preparation

### 1.1 Loading modules

In [1]:
import numpy as np  # Scientific computing
import pandas as pd # Data analysis and manipulation

from datetime import datetime # Dates and times manipulations

import requests # Send HTTP requests easily


# Plotly is a graphing library for interactive, publication-quality graphs
# pip install plotly==4.5.4
import plotly.graph_objects as go                   
import plotly.express as px
from plotly.subplots import make_subplots

### 1.2 Gathering the financial data using the Financial Modeling Prep API

In [2]:
# Creating a function to get the financial data by specifying the Ticker and the Financial Statement
def get_financials(ticker, financial_statement):
    
    # Sending an HTTP request to the Financial Modeling Prep API and convert it to a json() file 
    resp = requests.get('https://financialmodelingprep.com/api/v3/financials/{}/{}?period=quarter'.format(financial_statement, ticker)).json()
    
    # Extracting the financials (Dropping the symbol key)
    financials = resp['financials']
    
    # Setting up the diplay format of the floats in the pandas dataframes
    pd.options.display.float_format = '{:,}'.format
    
    # Creating the data frame using pandas from_dict() function
    financials = pd.DataFrame.from_dict(financials)
    
    # Converting the 'date' column to a datetime type using datetime
    financials['date'] = pd.to_datetime(financials['date'])

    # Setting the index to 'date'
    financials.set_index('date', inplace = True)

    # Converting the columns into Numeric values
    financials = financials.apply(pd.to_numeric, errors = 'coerce')
    
    return financials

In [33]:
# Choosing a ticker and a financial statement
ticker = 'IBM'
financial_statement = 'income-statement'

financials = get_financials(ticker, financial_statement)
financials.head()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin
date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,"21,777,000,000.0",0.0,"10,677,000,000.0","11,100,000,000.0","1,596,000,000.0","5,433,000,000.0","7,029,000,000.0","3,993,000,000.0","354,000,000.0","3,993,000,000.0",...,0.509712081554,0.25912660146,0.183358589337,0.16852642696400003,0.132020021123,"5,643,000,000.0","3,993,000,000.0","3,670,000,000.0",0.183358589337,0.16852642696400003
2019-09-30,"18,028,000,000.0",0.0,"9,692,000,000.0","8,335,000,000.0","1,553,000,000.0","5,024,000,000.0","6,577,000,000.0","1,522,000,000.0","432,000,000.0","1,522,000,000.0",...,0.462336365653,0.17700244064800003,0.08442422897710002,0.0927446194808,0.16812735744399998,"3,191,000,000.0","1,522,000,000.0","1,672,000,000.0",0.08442422897710002,0.0927446194808
2019-06-30,"19,160,000,000.0",0.0,"10,151,000,000.0","9,010,000,000.0","1,407,000,000.0","5,456,000,000.0","6,863,000,000.0","2,768,000,000.0","348,000,000.0","2,768,000,000.0",...,0.47025052192100003,0.212004175365,0.144467640919,0.13037578288100002,0.123068893528,"4,062,000,000.0","2,768,000,000.0","2,498,000,000.0",0.144467640919,0.13037578288100002
2019-03-31,"18,182,000,000.0",0.0,"10,139,000,000.0","8,043,000,000.0","1,433,000,000.0","4,691,000,000.0","6,124,000,000.0","1,883,000,000.0","210,000,000.0","1,883,000,000.0",...,0.442360576394,0.18309316906800002,0.10356396436000001,0.0875041249588,0.232097679023,"3,329,000,000.0","1,883,000,000.0","1,591,000,000.0",0.10356396436000001,0.0875041249588
2018-12-31,"21,761,000,000.0",0.0,"11,073,000,000.0","10,687,000,000.0","1,358,000,000.0","4,701,000,000.0","6,059,000,000.0","4,434,000,000.0","193,000,000.0","4,434,000,000.0",...,0.49110794540700003,0.25485961123100004,0.203759018427,0.0896558062589,0.153347732181,"5,546,000,000.0","4,434,000,000.0","1,951,000,000.0",0.203759018427,0.0896558062589


## 2 Income Statement Analysis

### Quarter

A quarter is a three-month period on a company's financial calendar that acts as a basis for periodic financial reports and the paying of dividends.

A quarter refers to one-fourth of a year and is typically expressed as "Q1” for the first quarter, “Q2” for the second quarter, and so forth.

### Revenue

Revenue is the income generated from normal business operations and includes discounts and deductions for returned merchandise. It is the top line or gross income figure from which costs are subtracted to determine net income.

**The cost of revenue :** is the total cost of manufacturing and delivering a product or service to consumers. 

Additional :

Cost of revenue is different from cost of goods sold (COGS) because the former also includes costs outside of production, such as distribution and marketing.

When comparing profit measures using a standard formula for profit margins such as those listed in an income statement, creating a profit margin measure based on the cost of revenue would generate a lower value than those typically used by corporations for quarterly reporting. The contribution margin includes total variable costs, and the gross margin only includes the COGS or the cost of services. A company with a low cost of revenue to total revenue percentage indicates that it is in stable financial health and may have strong sales.

**Gross profit :** is the profit a company makes after deducting the costs associated with making and selling its products, or the costs associated with providing its services.

**Research and development Expenses (R&D):**  are associated with the research and development of a company's goods or services. A company generally incurs R&D expenses in the process of finding and creating new products or services.

**Selling, general and administrative expenses (SG&A) :** are reported on the income statement as the sum of all direct and indirect selling expenses and all general and administrative expenses (G&A) of a company.

**Operating expense :** is an expense a business incurs through its normal business operations. Often abbreviated as OPEX, operating expenses include rent, equipment, inventory costs, marketing, payroll, insurance, step costs, and funds allocated for research and development.

**Interest expense :** is the cost incurred by an entity for borrowed funds. 

**Income tax expense :** is the amount of expense that a business recognizes in an accounting period for the government tax related to its taxable profit.

**Net income (NI) :** also called net earnings, is a useful number for investors to assess how much revenue exceeds the expenses of an organization. This number is an indicator of a company's profitability.

### 2.1 Plotting the Income statement of the last Quarter

In [34]:
# Creating a function that will setup our Waterfall chart
# Quaters are specified in an increasing order, last quarter is 0, the one before 1 and so on ...
def setup_waterfall_chart(financials, quarter):
    
    # Creating the list of columns that we will display
    columns = ["Revenue", "Cost of Revenue", "Gross Profit", "R&D Expenses", "SG&A Expense", "Operating Expenses", 'Interest Expense', 'Earnings before Tax', 'Income Tax Expense', 'Net Income']

    # Getting the first value from each column
    values = [financials[column].values[quarter] for column in columns]

    # Transforming the costs, expenses and taxes into their negative values
    values = [- value if count in [1, 3, 4, 6, 8] else value for (count, value) in enumerate(values)]

    # Creating the text list of each column for the waterfall chart
    text = [f'{value / 10**9} B' for value in values] 

    # Setting up the measures
    measures = ["relative", "relative", "total", "relative", "relative", "total", "realtive", "total", "relative", "total"]
    
    return measures, columns, values, text

In [35]:
# Getting the values text of the last quarter financials
measures, columns, values, text = setup_waterfall_chart(financials, 0)

# Creating the waterfall chart of the last quarter profit and loss statement
fig = go.Figure(go.Waterfall(
    name = "20", orientation = "v",
    measure = measures,
    x = columns,
    textposition = "outside",
    text = text ,
    y = values,
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
        title = "Income statement Last Quarter for {}".format(ticker),
        showlegend = True,
        yaxis_title="USD ($)",
)

fig.show()

### 2.2 Revenue Analysis

#### Revenue Growth

Revenue growth is the increase (or decrease) in a company’s sales from one period to the next. 

Shown as a percentage, revenue growth illustrates the increases and decreases over time identifying trends in the business.

In [36]:
financials['Revenue Growth'].values

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.19722734,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.15547823,  0.0135914 , -0.1331    ])

This Ratio is **not calculated**, in our financial data, let's fix it. We are going to calculated the Revenue growth as :

$$ \cfrac{RevenueQuarter_{n} - Revenue Quarter_{n - 1}} {Revenue Quarter_{n - 1}}  $$

In [37]:
# chekcing for errors (zero values) in the financials Revenue column
0 in financials['Revenue'].values

True

In [38]:
# There is a null values, It corresponds to the date(s)
financials[financials['Revenue'].values == 0]

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin
date,,,,,,,,,,,,,,,,,,,,,
2014-04-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We see that the row is empty, but shall we already drop it ? 

**No** ! we have to determine the reality of the problem
- Is it a typing error ?
- Will it create another error if we drop ?

Lets plot the revenue to have a better idea

In [39]:
# Plotting the Revenue of IBM per Date
fig = px.line(financials, x=financials.index, y=financials['Revenue'])
fig.update_layout(title = "False Revenue of {} per Date".format(ticker), showlegend = True, yaxis_title="USD ($)", xaxis_title="Date")
fig.show()

In [40]:
financials.loc['2014']

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin
date,,,,,,,,,,,,,,,,,,,,,
2014-12-31,"24,113,000,000.0",0.0,"11,251,000,000.0","12,862,000,000.0","1,320,000,000.0","6,034,000,000.0","7,354,000,000.0","7,095,000,000.0","117,000,000.0","7,095,000,000.0",...,0.533405217103,0.337618711898,0.294239621781,0.22738771616999998,0.212001824742,"8,141,000,000.0","7,095,000,000.0","5,483,000,000.0",0.294239621781,0.22738771616999998
2014-09-30,"21,832,000,000.0",0.0,"10,805,000,000.0","11,027,000,000.0","1,159,000,000.0","5,254,000,000.0","6,413,000,000.0","4,740,000,000.0","127,000,000.0","4,740,000,000.0",...,0.505084279956,0.269283620374,0.21711249542,0.0008244778307069999,0.131366801026,"5,879,000,000.0","4,740,000,000.0","18,000,000.0",0.21711249542,0.0008244778307069999
2014-06-30,"46,848,000,000.0",0.0,"24,330,000,000.0","22,518,000,000.0","2,958,000,000.0","11,892,000,000.0","14,850,000,000.0","8,151,000,000.0","240,000,000.0","8,151,000,000.0",...,0.480660860656,0.223232581967,0.17398821721299998,0.139194842896,0.10988729508199999,"10,458,000,000.0","8,151,000,000.0","6,521,000,000.0",0.17398821721299998,0.139194842896
2014-04-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-03-31,"22,236,000,000.0",-0.197227336727,"11,608,000,000.0","10,628,000,000.0","1,402,000,000.0","6,272,000,000.0","7,340,000,000.0","3,288,000,000.0","105,000,000.0","3,037,000,000.0",...,0.478,0.193,0.1413,0.107,0.1097,"4,283,000,000.0","3,142,000,000.0","2,384,000,000.0",0.1366,0.1072


#### Analysing the results

As we can see the problem comes from two dates, April 2014 and Jun 2014. The values of the columns must have been added together. 

We can divid the values of Jun 2014 by two and copy it in the April 2014 row.

In [41]:
financials.loc['2014-06-30', : 'Earnings before Tax'] /= 2
financials.loc['2014-06-30', ['Dividend per Share', 'EBITDA', 'EBIT']] /= 2
financials['2014-04-29'] = financials['2014-06-30'].values

financials['2014']

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,EBITDA,EBIT,Consolidated Income,Earnings Before Tax Margin,Net Profit Margin
date,,,,,,,,,,,,,,,,,,,,,
2014-12-31,"24,113,000,000.0",0.0,"11,251,000,000.0","12,862,000,000.0","1,320,000,000.0","6,034,000,000.0","7,354,000,000.0","7,095,000,000.0","117,000,000.0","7,095,000,000.0",...,0.533405217103,0.337618711898,0.294239621781,0.22738771616999998,0.212001824742,"8,141,000,000.0","7,095,000,000.0","5,483,000,000.0",0.294239621781,0.22738771616999998
2014-09-30,"21,832,000,000.0",0.0,"10,805,000,000.0","11,027,000,000.0","1,159,000,000.0","5,254,000,000.0","6,413,000,000.0","4,740,000,000.0","127,000,000.0","4,740,000,000.0",...,0.505084279956,0.269283620374,0.21711249542,0.0008244778307069999,0.131366801026,"5,879,000,000.0","4,740,000,000.0","18,000,000.0",0.21711249542,0.0008244778307069999
2014-06-30,"23,424,000,000.0",0.0,"12,165,000,000.0","11,259,000,000.0","1,479,000,000.0","5,946,000,000.0","7,425,000,000.0","4,075,500,000.0","120,000,000.0","4,075,500,000.0",...,0.480660860656,0.223232581967,0.17398821721299998,0.139194842896,0.10988729508199999,"5,229,000,000.0","4,075,500,000.0","6,521,000,000.0",0.17398821721299998,0.139194842896
2014-04-29,"23,424,000,000.0",0.0,"12,165,000,000.0","11,259,000,000.0","1,479,000,000.0","5,946,000,000.0","7,425,000,000.0","4,075,500,000.0","120,000,000.0","4,075,500,000.0",...,0.480660860656,0.223232581967,0.17398821721299998,0.139194842896,0.10988729508199999,"5,229,000,000.0","4,075,500,000.0","6,521,000,000.0",0.17398821721299998,0.139194842896
2014-03-31,"22,236,000,000.0",-0.197227336727,"11,608,000,000.0","10,628,000,000.0","1,402,000,000.0","6,272,000,000.0","7,340,000,000.0","3,288,000,000.0","105,000,000.0","3,037,000,000.0",...,0.478,0.193,0.1413,0.107,0.1097,"4,283,000,000.0","3,142,000,000.0","2,384,000,000.0",0.1366,0.1072


#### Caculating the Revenue Growth

In [42]:
for i in range(len(financials) - 1):
    financials['Revenue Growth'][i] = (financials['Revenue'][i] - financials['Revenue'][i + 1]) / financials['Revenue'][i + 1]
    
financials['Revenue Growth'].values[:5]

array([ 0.20795429, -0.05908142,  0.05378946, -0.16446854,  0.16027726])

### Plotting the results

In [43]:
# Making a plotly subplot
fig = make_subplots(rows=2, cols=1, subplot_titles=(f'Revenue of {ticker}', f'Revenue Growth of {ticker}'))

# Adding the Revenue subplot
fig.add_trace(go.Scatter(x=financials.index, y=financials['Revenue'], name = 'Revenue'), row=1, col=1)

# Adding the Revenue Growth subplot
fig.add_trace(go.Scatter(x=financials.index, y=financials['Revenue Growth'], name = 'Revenue Growth'), row=2, col=1)

fig.update_yaxes(title_text='USD ($)', row=1, col=1)

fig.show()

#### Analysing the results

We observe from the first graph that : 
- The revenue of IBM was increasing from 2009 to its peak in decembre 2011 with **29.486B USD $\$ $**.
- Then he started falling until March 2015 where it's stabilized with a **20B USD$\$ $** mean.

Its annual fluctuaction can be observed from the second graph, we see a repeated pattern each year. It can be interpreted based on the quarters $Q_i$ :
- $Q_1$ - Mar : the minimum value of the revenue growth for each year ;
- $Q_2$ - Jun : a primal increase in the revenue growth ;
- $Q_3$ - Sep : a small decrease in the revenue growth ;
- $Q_4$ - Dec : a second increase to its peak value of the year, followed by a decrease and the a repetition of the pattern.

### 2.3 Margin Analysis

 #### Gross Margin

This is revenues minus the cost of goods sold (COGS), divided by revenues.

It indicates the amount of money earned from the sale of goods and services, before selling and administrative charges are considered. 

In essence, it reveals the ability of an organization to earn a reasonable return on its offerings.

In our study we took the cost of revenu instead of the COGS, the profit margin measure will generate a lower value than those typically used by corporations for quarterly reporting.

In [44]:
# For the Last Quarter
gross_margin = round(financials["Gross Margin"][0] * 100, 2)
print(f' The Gross Margin of IBM for the last Quarter : {gross_margin}%')

 The Gross Margin of IBM for the last Quarter : 50.97%


#### Operating Margin

This is the profit earned after all operating expenses have been subtracted from the gross margin, divided by revenues. 

It reveals the amount that a business has earned before financing and other costs are considered.

It is also called the EBIT Margin (earnings before interest and taxes).

#### Net Profit Margin

This is the profit earned after all operating and non-operating costs have been subtracted from the gross margin, divided by revenues. 

This is the ultimate analysis item - can a business earn a profit when all deductions are considered?


### Plotting the results

In [45]:
# Making a plotly subplot
fig = make_subplots(rows=3, cols=1, subplot_titles=(f'Gross Margin of {ticker}', f'Operating Margin of {ticker}', f'Net Profit Margin of {ticker}'))

# Adding the Gross Profit Margin subplot
fig.add_trace(go.Scatter(x=financials.index, y=financials['Gross Margin'], name = 'Gross Margin'), row=1, col=1)

# Adding the EBITDA Margin subplot
fig.add_trace(go.Scatter(x=financials.index, y=financials['EBIT Margin'], name = 'Operating Margin'), row=2, col=1)

## Adding the Net Profit Margin subplot
fig.add_trace(go.Scatter(x=financials.index, y=financials['Net Profit Margin'], name = 'Net Profit Margin'), row=3, col=1)


fig.show()

### Analysing the results

A constructive analysis of margins should be made between the major competitors in an industry. 

We will not be performing it in this notebook, a thorough analysis has been made by Jawaher Alotaibi and can be found on : https://www.researchgate.net/publication/309533206_Financial_Analysis_of_IBM

### Ressources

- Financial Modeling Prep API Documentation : https://financialmodelingprep.com/developer/docs/ 
- Plotly financial charts : https://plotly.com/python/financial-charts/
- IBM financial analysis : https://www.researchgate.net/publication/309533206_Financial_Analysis_of_IBM
- Financial Definitions : 
    - https://www.investopedia.com/
    - https://en.wikipedia.org/wiki/Ticker_symbol


- The main Resources :   
    - https://www.thebalance.com/investing-lesson-4-income-statement-analysis-357580
    - https://www.accountingtools.com/articles/income-statement-analysis.html
    - https://codingandfun.com/

### Created by MEBREK Brahim